In [11]:
from Model import *
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

## Define batch size, number of classes, and image dimensions

In [12]:
batch_size = 1024
num_classes = 200
im_rows,im_cols,im_channels = 64,64,3

## Load Train and Test sets

In [13]:
class Load_data:
    def __init__(self,dirpath=r'data\ds1\CUB_200_2011\images\elmira'):
        self.dirpath=dirpath
        self.ds_x=[]
        
    def make_ds(self):
        
        classes=[x[0] for x in os.walk(self.dirpath)]
        classes = classes[1:]  # remove parent dir
        self.ds_y=[]
        #print(classes)
        for j in range(len(classes)):
            images_curr_class = os.listdir(classes[j])
            for i in range(len(images_curr_class)):
                
                img=Image.open(os.path.join(classes[j], images_curr_class[i])).resize(size=(64,64), resample=Image.BICUBIC)
                img=np.array(img)
                #print(img.shape)
                self.ds_x.append(img)
                
                #curr_y=np.zeros(shape=(len(classes)))
                #curr_y[j]=1
                #self.ds_y.append(curr_y)
                self.ds_y.append(j)
                

        self.ds_y=np.array(self.ds_y)
        #print("y",self.ds_y.shape)
        #self.ds_x=np.concatenate( self.ds_x[1:], axis=0 ).reshape(-1,64,64,3)
        #print("x",self.ds_x.shape)  
        
        return self.ds_x,self.ds_y
        
     
        
                 
        

In [14]:
ld_tr=Load_data(dirpath=r'data\ds1\CUB_200_2011\images\train')
x_train,t_train=ld_tr.make_ds()

tr = np.empty((len(x_train), 64,64,3))
for i in range(len(x_train)):
    if np.sum(x_train[i].shape)==131:
        tr[i,:,:,:]=x_train[i]
x_train=tr
print(x_train.shape,t_train.shape)

(7470, 64, 64, 3) (7470,)


In [15]:
ld_te=Load_data(dirpath=r'data\ds1\CUB_200_2011\images\test')
x_test,t_test=ld_te.make_ds()

te = np.empty((len(x_test), 64,64,3))
for i in range(len(x_test)):
    if np.sum(x_test[i].shape)==131:
        te[i,:,:,:]=x_test[i]
x_test=te
print(x_test.shape,t_test.shape)

(4318, 64, 64, 3) (4318,)


## Reshape and normalize dataset

In [16]:
x_train = x_train.reshape(-1,3,im_rows,im_cols).astype(float)
x_test = x_test.reshape(-1,3,im_rows,im_cols).astype(float)
x_train /= 255
x_train= x_train - np.mean(x_train)
x_test /= 255
x_test= x_test -np.mean(x_test)
x_train.shape

(7470, 3, 64, 64)

## Define the layers of our convolutional neural network

In [17]:
input_layer = Input("Input",(3,im_rows,im_cols),1024)
c1 = Conv2D(stride=(1,1),filter_shape=(5,5),n_filters=20,padding=(0,0),activation="relu")
mp1 = MaxP((2,2),(2,2))
c2 = Conv2D(stride=(1,1),filter_shape=(5,5),n_filters=50,padding=(0,0),activation="relu")
mp2 = MaxP((2,2),(2,2))
fc1 = FC("relu",500)
fc2 = FC("relu",200)
loss = CELoss()

## Binding the layers together to build the network

In [18]:
graph = c1(input_layer)
graph = mp1(graph)
graph = c2(graph)
graph = mp2(graph)
graph = fc1(graph)
graph = fc2(graph)
graph = loss(graph,input_layer)

In [19]:
model = Model(graph,x_train,t_train, lr=0.01) # lr is the learning rate

Initing


## Train the network

In [ ]:
losses=[]
acc=[]
for i in range(10):
    model.forward_pass()
    model.backward_pass()
    model.update()
    losses.append(np.mean(loss.fw_result))
    acc.append(len([x for x in np.argmax(fc2.fw_result,axis=1) - input_layer.target if x == 0])/batch_size)
    print("i = {} Error = {} Acc_batch = {}".format(i,np.mean(loss.fw_result),len([x for x in np.argmax(fc2.fw_result,axis=1) - input_layer.target if x == 0])/batch_size))

c:\users\ehajimani\appdata\local\programs\python\python36\lib\site-packages\skimage\util\shape.py:247: RuntimeWarning: Cannot provide views on a non-contiguous input array without copying.
  warn(RuntimeWarning("Cannot provide views on a non-contiguous input "


1024
i = 0 Error = 5.712828018908574 Acc_batch = 0.0
1024
i = 1 Error = 5.29839068880427 Acc_batch = 0.0
1024


C:\Users\ehajimani\Documents\Eduardo\exercise_2\Exercise_2\Loss.py:40: RuntimeWarning: divide by zero encountered in log
  self.fw_result = -np.log(self.prob[range(n_pred), y_train])


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(losses) 
plt.ylabel('Error')
plt.xlabel('Iterations')
plt.show()

In [ ]:
plt.plot(acc) 
plt.ylabel('Accuracy')
plt.xlabel('Iterations')
plt.show()